In [1]:
import pandas as pd

In [7]:
user_account = pd.read_csv('raw/userID_accountID.csv')
user_card = pd.read_csv('raw/userID_cardID.csv')
user_device = pd.read_csv('raw/userid_deviceid.csv')

In [8]:
user_account

,userID,accountID
0,200221000026960,200221100002264
1,191206720001042,191206100000350
2,191210484000661,191210100000238
3,190709000001059,190707100001431
4,191227000009403,191227100000397
...,...,...
10748,191223889025698,191223100001747
10749,200429000002522,200720100000957
10750,191220734006017,191220100001326
10751,200104000043475,200629100000340


In [9]:
user_account.duplicated().value_counts()

False    10753
dtype: int64

In [10]:
user_card.duplicated().value_counts()

False    51131
dtype: int64

In [12]:
user_card

,userID,cardID
0,191111735008352,191111000016864
1,180818000008076,200524000004399
2,190120000001593,190612000556786
3,191124227003671,191124000007604
4,191012537001139,191012000000490
...,...,...
51126,190924825001895,190621000812296
51127,191013325000140,190612000243670
51128,191119199000747,191119000007075
51129,190123000017769,190612000644977


In [11]:
user_device.duplicated().value_counts()

False    218393
dtype: int64

In [19]:
user_device

,userID,deviceID
0,190215000014037,5e34bdd5266ff7c6
1,190313000016148,1761c1b0d746b8e8
2,190318000000574,14e0d55eb848e527
3,190722000001381,1f6b129e317e35fa
4,191016219002578,0e2fa6b350f0f020
...,...,...
218388,190502000001734,09c66ac813712993
218389,190801000001894,e11920dbaf0f9d2
218390,191031382014740,25715da6516d4853
218391,191225960015857,a33aa775c5c484b8


In [26]:
total_users = user_account['userID'].tolist() + user_device['userID'].tolist() + user_card['userID'].tolist()

In [27]:
len(total_users)

280277

In [31]:
pd.Series(total_users, name="userID").to_csv('./user_list.csv', index=False)

In [33]:
user_device.columns = ['source', 'target']
user_device.to_csv('./user_device.csv', index=False)